**Data Description**

*there are two tables:*

2022_accidents_vehicles - List of vehicles involved in accidents handled by the police in the city of Barcelona . 
An accident may be associated with more than one vehicle. Including whether the cause is pedestrian , vehicle type , model , brand, color and license type and age of the person who was driving. I consider this table should contain full list of accidents

2022_accidents_cause_driver - Accidents by driver cause managed by the Guàrdia Urbana in the city of Barcelona. It will be used to define a reason for an accident



In [1]:
import sqlite3
import pandas as pd


In [2]:
conn = sqlite3.connect(':memory:')


In [8]:
# Loading data into a pandas DataFrame for 2022_accidents_vehicles
file_path = "2022_accidents_vehicles_gu_bcn.csv"
df = pd.read_csv(file_path)

# Create a table in the SQLite database
df.to_sql('2022_accidents_vehicles', conn, if_exists='replace', index=False)



14906

In [9]:

# Loading data into a pandas DataFrame for 2022_accidents_cause_driver
file_path = "2022_accidents_causa_conductor_gu_bcn_.csv"
df = pd.read_csv(file_path)

# Create a table in the SQLite database
df.to_sql('2022_accidents_cause_driver', conn, if_exists='replace', index=False)


9320

**validate number of unique cases in 2022_accidents_cause_driver**

In [13]:
import pandas as pd
import sqlite3

# Assuming you have an SQLite connection named `conn`
# I will have to use this query to be able to present query results using Jupiter
query = """
SELECT 
  COUNT(DISTINCT Numero_expedient) AS dist_case,
  COUNT(Numero_expedient) AS total_vehicles
FROM "2022_accidents_vehicles";
"""

result = pd.read_sql_query(query, conn)
result.head()



,dist_case,total_vehicles
0,7984,14901


**validate number of unique cases in 2022_accidents_cause_driver**


In [14]:
query = """
SELECT 
  COUNT (DISTINCT Numero_expedient) AS dist_case,
  COUNT(Numero_expedient)  AS total_vehicles

FROM "2022_accidents_cause_driver";
"""

result = pd.read_sql_query(query, conn)
result.head()

,dist_case,total_vehicles
0,7989,9320


**validate what are the examples where there is more then one reason for the case**


In [19]:
query = """
SELECT Numero_expedient, Descripcio_causa_mediata  FROM
(SELECT *,
   Numero_expedient, 
   Descripcio_causa_mediata,
   COUNT(1) OVER(PARTITION BY Numero_expedient) as cnt
  
FROM "2022_accidents_cause_driver")
WHERE cnt > 1
LIMIT 4
"""

result = pd.read_sql_query(query, conn)
result.head()

,Numero_expedient,Descripcio_causa_mediata
0,2022S000006,Manca d'atenció a la conducció
1,2022S000006,No respectar distàncies
2,2022S000014,Manca d'atenció a la conducció
3,2022S000014,Altres


*I can see from here that sometimes there is more than one cause (up to 4 causes per case is defined)*

In [21]:
query = """


SELECT COUNT (distinct Numero_expedient) FROM
(SELECT *,
  Numero_expedient, 
  Descripcio_causa_mediata,
  COUNT(1) OVER(PARTITION BY Numero_expedient) as cnt
  
FROM "2022_accidents_cause_driver")
WHERE cnt > 1 

"""

result = pd.read_sql_query(query, conn)
result.head()

,COUNT (distinct Numero_expedient)
0,1238


*1238 cases when there are multiple causes while total number of cases is 7206, which is 17% from total number of cases, it means we can't neglect this information.But going forward we will use first occurance reason for the research .*

**Now I check if all cases presented in vehicle table are also available in full list of cases**

In [25]:
query = """

SELECT * 
FROM "2022_accidents_vehicles"
WHERE NOT Exists (select 1
                    FROM "2022_accidents_cause_driver"
                    WHERE "2022_accidents_cause_driver".Numero_expedient= "2022_accidents_vehicles".Numero_expedient);
 
"""

result = pd.read_sql_query(query, conn)
result.head()

,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal,Descripcio_dia_setmana,NK_Any,...,Descripcio_tipus_vehicle,Descripcio_model,Descripcio_marca,Descripcio_color,Descripcio_carnet,Antiguitat_carnet,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud,Latitud
0,None,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",163504,Isabel II / Correus ...,0002 0002,Dilluns,2022,...,None,None,None,None,None,None,431713.95,4581617.07,2.182206,41.381336
1,None,3,Sants-Montjuïc,16,la Bordeta,30602,None,0081 0111,Dilluns,2022,...,None,None,None,None,None,None,427554.79,4580367.12,2.132619,41.369714
2,None,5,Sarrià-Sant Gervasi,26,Sant Gervasi - Galvany,344101,Gràcia ...,0085 0085,Divendres,2022,...,None,None,None,None,None,None,429147.62,4583418.77,2.151306,41.397341
3,None,7,Horta-Guinardó,35,el Guinardó,194406,Maragall / Pinar del Río ...,0167 0169,Dissabte,2022,...,None,None,None,None,None,None,431457.60,4586212.43,2.178619,41.422703
4,None,9,Sant Andreu,58,Baró de Viver,314100,Santa Coloma ...,0114 0114,Dilluns,2022,...,None,None,None,None,None,None,433320.41,4589051.71,2.200596,41.448433


*as a result I can see a few records with NULL case number, I will remove them, but before removing. I will create a back up copy of all tables*


In [27]:
query = """
CREATE TABLE "as_is_2022_accidents_vehicles" AS
SELECT * FROM "2022_accidents_vehicles";
"""

conn.execute(query)


*copy of the vehicle table is created*

**Now I will check number of records before deleting records (it is 5 records I am planning to delete . total there are 14906 records , it means after deletion my number of records will be 14901)**


In [28]:
query = """

 
SELECT * 
FROM "2022_accidents_vehicles"
WHERE NOT Exists (select 1
                    FROM "2022_accidents_cause_driver"
                    WHERE "2022_accidents_cause_driver".Numero_expedient= "2022_accidents_vehicles".Numero_expedient);

 
 
 
"""

result = pd.read_sql_query(query, conn)
result.head()

,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal,Descripcio_dia_setmana,NK_Any,...,Descripcio_tipus_vehicle,Descripcio_model,Descripcio_marca,Descripcio_color,Descripcio_carnet,Antiguitat_carnet,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud,Latitud
0,None,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",163504,Isabel II / Correus ...,0002 0002,Dilluns,2022,...,None,None,None,None,None,None,431713.95,4581617.07,2.182206,41.381336
1,None,3,Sants-Montjuïc,16,la Bordeta,30602,None,0081 0111,Dilluns,2022,...,None,None,None,None,None,None,427554.79,4580367.12,2.132619,41.369714
2,None,5,Sarrià-Sant Gervasi,26,Sant Gervasi - Galvany,344101,Gràcia ...,0085 0085,Divendres,2022,...,None,None,None,None,None,None,429147.62,4583418.77,2.151306,41.397341
3,None,7,Horta-Guinardó,35,el Guinardó,194406,Maragall / Pinar del Río ...,0167 0169,Dissabte,2022,...,None,None,None,None,None,None,431457.60,4586212.43,2.178619,41.422703
4,None,9,Sant Andreu,58,Baró de Viver,314100,Santa Coloma ...,0114 0114,Dilluns,2022,...,None,None,None,None,None,None,433320.41,4589051.71,2.200596,41.448433


**deleting records**


In [30]:
query = """
DELETE 
FROM "as_is_2022_accidents_vehicles"
WHERE Numero_expedient IN (
    SELECT Numero_expedient
    FROM "as_is_2022_accidents_vehicles"
    GROUP BY Numero_expedient
    HAVING COUNT(*) > 1
);
"""

cursor = conn.cursor()
cursor.execute(query)
conn.commit()


**Entire table is on Catalan language, I will update all column namew with clear definition in English**


In [32]:
columns_to_rename = [
    ('Numero_expedient', 'case_number'),
    ('Codi_districte', 'district_code'),
    ('Nom_districte', 'district_name'),
    ('Codi_barri', 'neighborhood_code'),
    ('Nom_barri', 'neighborhood_name'),
    ('Codi_carrer', 'street_code'),
    ('Nom_carrer', 'street_name'),
    ('Num_postal', 'postal_code'),
    ('Descripcio_dia_setmana', 'weekday'),
    ('NK_Any', 'year'),
    ('Mes_any', 'month_number'),
    ('Dia_mes', 'day_number'),
    ('Hora_dia', 'hour'),
    ('Descripcio_torn', 'part_of_the_day'),
    ('Descripcio_causa_vianant', 'pedestrian_cause_description'),
    ('Descripcio_tipus_vehicle', 'vehicle_type'),
    ('Descripcio_carnet', 'license_type'),
    ('Antiguitat_carnet', 'license_age')
]

cursor = conn.cursor()

for old_col, new_col in columns_to_rename:
    query = f'ALTER TABLE "2022_accidents_vehicles" RENAME COLUMN {old_col} TO {new_col};'
    cursor.execute(query)

conn.commit()


**validate if the result as expected**


In [35]:
query = """

 SELECT *
FROM "2022_accidents_vehicles"
LIMIT 3
 
"""

result = pd.read_sql_query(query, conn)
result.head()

,case_number,district_code,district_name,neighborhood_code,neighborhood_name,street_code,street_name,postal_code,weekday,year,...,vehicle_type,Descripcio_model,Descripcio_marca,Descripcio_color,license_type,license_age,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud,Latitud
0,2022S000792,-1,Desconegut,-1,Desconegut,-1,Elisabeth Eidenbenz ...,8-24,Dijous,2022,...,Taxi,TOYOTA PRIUS PLUS,TOYOTA,Negre/Groc,B,20,426794.73,4581925.76,2.123343,41.383684
1,2022S000792,-1,Desconegut,-1,Desconegut,-1,Elisabeth Eidenbenz ...,None,Dijous,2022,...,Turisme,LEON ST,SEAT,Gris,B,23,426794.73,4581925.76,2.123343,41.383684
2,2022S001565,-1,Desconegut,-1,Desconegut,-1,Sant Cugat ...,49-55,Dijous,2022,...,Bicicleta,Desconegut,Desconegut,Negre,B,17,426417.43,4586500.99,2.118274,41.424856


**Update weekday names to english names**


In [37]:
query = """

 
UPDATE "2022_accidents_vehicles"
SET `weekday` = 
    CASE 
        WHEN `weekday` = 'Dilluns' THEN 'Monday'
        WHEN `weekday` = 'Dimarts' THEN 'Tuesday'
        WHEN `weekday` = 'Dissabte' THEN 'Saturday'
        WHEN `weekday` = 'Dimecres' THEN 'Wednesday'
        WHEN `weekday` = 'Dijous' THEN 'Thursday'
        WHEN `weekday` = 'Diumenge' THEN 'Sunday'
        WHEN `weekday` = 'Divendres' THEN 'Friday'
        ELSE `weekday`
    END
  WHERE weekday is not null;
 
 
"""
cursor = conn.cursor()
cursor.execute(query)
conn.commit()

*I make validation of original table as is vs new table to ensure my weekday update correctly performed ! yes it looks great*

In [38]:
query = """

 
SELECT
   DISTINCT   table1.first_occurrence_number,   table2.first_occurrence_number
FROM (
  SELECT
    case_number as id,
    MIN(weekday) AS first_occurrence_number
  FROM
    "2022_accidents_vehicles"
  GROUP BY
    id
) AS table1
JOIN (
  SELECT
    Numero_expedient as id,
    MIN(Descripcio_dia_setmana) AS first_occurrence_number
  FROM
    "as_is_2022_accidents_vehicles"
  GROUP BY
    id
) AS table2
ON
  table1.id = table2.id
 
 
 
"""

result = pd.read_sql_query(query, conn)
result.head()

,first_occurrence_number,first_occurrence_number
0,Sunday,Diumenge
1,Monday,Dilluns
2,Tuesday,Dimarts
3,Wednesday,Dimecres
4,Thursday,Dijous


**Update the pedestrian_cause_description with english description**


In [41]:
query = """

UPDATE "2022_accidents_vehicles"
SET `pedestrian_cause_description` = 
    CASE 
        WHEN `pedestrian_cause_description` = 'No és causa del  vianant' THEN 'It is not the cause of the pedestrian'
        WHEN `pedestrian_cause_description` = 'Transitar a peu per la calçada' THEN 'Walk along the road'
        WHEN `pedestrian_cause_description` = 'Altres' THEN 'Others'
        WHEN `pedestrian_cause_description` = 'Creuar per fora pas de vianants' THEN 'Cross over the pedestrian crossing'
        WHEN `pedestrian_cause_description` = 'Desobeir el senyal del semàfor' THEN 'Disobey the traffic light signal'
        WHEN `pedestrian_cause_description` = 'Desobeir altres senyals' THEN 'Disobey the other signal'
        ELSE `pedestrian_cause_description`
    END
  WHERE pedestrian_cause_description is not null;

 
 
 
"""
cursor = conn.cursor()
cursor.execute(query)
conn.commit()

 **Update english description of vehicle types**

In [42]:
query = """

 UPDATE "2022_accidents_vehicles"
SET `vehicle_type` = 
    CASE 
        WHEN `vehicle_type` = 'Tot terreny' THEN 'Off-road vehicle'
        WHEN `vehicle_type` = 'Bicicleta' THEN 'Bicycle'
        WHEN `vehicle_type` = 'Motocicleta' THEN 'motorcycle'
        WHEN `vehicle_type` = 'Furgoneta' THEN 'Van'
        WHEN `vehicle_type` = 'Tractor camió' THEN 'Tractor truck'
        WHEN `vehicle_type` = 'Veh. mobilitat personal sense motor' THEN 'vehicle personal mobility without motor'
        WHEN `vehicle_type` = 'Camió rígid > 3,5 tones' THEN 'Rigid truck > 3.5 tonnes'
        WHEN `vehicle_type` = 'Veh. mobilitat personal amb motor' THEN 'vehicle personal mobility with motor'
        WHEN `vehicle_type` = 'Ciclomotor' THEN 'moped'
        WHEN `vehicle_type` = 'Camió rígid <= 3,5 tones' THEN 'Rigid truck <= 3.5 tons'
        WHEN `vehicle_type` = 'Autobús' THEN 'bus'
        WHEN `vehicle_type` = 'Autobús articulat' THEN 'Articulated bus'
        WHEN `vehicle_type` = 'Desconegut' THEN 'Unknown'
        WHEN `vehicle_type` = "Maquinària d'obres i serveis" THEN 'Construction machinery and services'
        WHEN `vehicle_type` = 'Autocar' THEN 'long distance bus'
        WHEN `vehicle_type` = 'Altres vehicles amb motor' THEN 'Other motor vehicles'
        WHEN `vehicle_type` = 'Tren o tramvia' THEN 'Train or tram'
        WHEN `vehicle_type` = 'Tricicle' THEN 'tricycle'
        WHEN `vehicle_type` = 'Altres vehicles sense motor' THEN 'Other non-motorized vehicles'
        ELSE `vehicle_type`
    END
  WHERE vehicle_type is not null;
 
 
 
"""

cursor = conn.cursor()
cursor.execute(query)
conn.commit()

**Now I want to add a new column Descripcio_causa_mediata from another data set to know what is the reason for every case**

In [45]:
query = """
ALTER TABLE "2022_accidents_vehicles"
ADD COLUMN descripcio_causa_mediata STRING;
"""

cursor = conn.cursor()
cursor.execute(query)
conn.commit()


<class 'sqlite3.OperationalError'>: duplicate column name: descripcio_causa_mediata

*Valdate table structure, see, new column was added*

In [48]:
result = pd.read_sql_query('PRAGMA table_info("2022_accidents_vehicles");', conn)
result


,cid,name,type,notnull,dflt_value,pk
0,0,case_number,TEXT,0,None,0
1,1,district_code,INTEGER,0,None,0
2,2,district_name,TEXT,0,None,0
3,3,neighborhood_code,INTEGER,0,None,0
4,4,neighborhood_name,TEXT,0,None,0
5,5,street_code,INTEGER,0,None,0
6,6,street_name,TEXT,0,None,0
7,7,postal_code,TEXT,0,None,0
8,8,weekday,TEXT,0,None,0
9,9,year,INTEGER,0,None,0


*Now I update new column based on first occurred record in full list of acidents. there are 15% cases when there is more than 1 reason, I only consider first occured! this is a valid assumption for our purpose*



In [50]:
query = """
UPDATE "2022_accidents_vehicles"
SET Descripcio_causa_mediata = (
  SELECT MIN(Descripcio_causa_mediata)
  FROM "2022_accidents_cause_driver"
  WHERE "2022_accidents_vehicles".case_number = "2022_accidents_cause_driver".Numero_expedient
);
"""

cursor = conn.cursor()
cursor.execute(query)
conn.commit()


*Now I check if table is succesfully updated*

In [52]:
query = """

SELECT * FROM "2022_accidents_vehicles" LIMIT 100
  
"""

result = pd.read_sql_query(query, conn)
result.head()

,case_number,district_code,district_name,neighborhood_code,neighborhood_name,street_code,street_name,postal_code,weekday,year,...,Descripcio_model,Descripcio_marca,Descripcio_color,license_type,license_age,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud,Latitud,descripcio_causa_mediata
0,2022S000792,-1,Desconegut,-1,Desconegut,-1,Elisabeth Eidenbenz ...,8-24,Thursday,2022,...,TOYOTA PRIUS PLUS,TOYOTA,Negre/Groc,B,20,426794.73,4581925.76,2.123343,41.383684,Manca d'atenció a la conducció
1,2022S000792,-1,Desconegut,-1,Desconegut,-1,Elisabeth Eidenbenz ...,None,Thursday,2022,...,LEON ST,SEAT,Gris,B,23,426794.73,4581925.76,2.123343,41.383684,Manca d'atenció a la conducció
2,2022S001565,-1,Desconegut,-1,Desconegut,-1,Sant Cugat ...,49-55,Thursday,2022,...,Desconegut,Desconegut,Negre,B,17,426417.43,4586500.99,2.118274,41.424856,Avançament defectuós/improcedent
3,2022S001565,-1,Desconegut,-1,Desconegut,-1,Sant Cugat ...,None,Thursday,2022,...,Desconegut,Desconegut,Vermell,A,2,426417.43,4586500.99,2.118274,41.424856,Avançament defectuós/improcedent
4,2022S001568,-1,Desconegut,-1,Desconegut,-1,Pallars ...,279-285,Thursday,2022,...,JET 14,SYM,Blau,B,10,433391.56,4584170.99,2.201986,41.404479,Manca precaució incorporació circulació


**Now I compare values , and I can see the values in both tables are equal , succefully joint and moved**


In [53]:
query = """

 
 
SELECT DISTINCT "2022_accidents_vehicles".case_number, min("2022_accidents_vehicles".descripcio_causa_mediata) as f0, min("2022_accidents_cause_driver".descripcio_causa_mediata) as f1
FROM "2022_accidents_vehicles"
JOIN 
"2022_accidents_cause_driver"
ON
"2022_accidents_vehicles".case_number ="2022_accidents_cause_driver".Numero_expedient

GROUP BY case_number
 
 
"""

result = pd.read_sql_query(query, conn)
result.head()

,case_number,f0,f1
0,2022S000001,Manca d'atenció a la conducció,Manca d'atenció a la conducció
1,2022S000002,Gir indegut o sense precaució,Gir indegut o sense precaució
2,2022S000003,Envair calçada contrària,Envair calçada contrària
3,2022S000004,No respectar distàncies,No respectar distàncies
4,2022S000005,Altres,Altres


**Now I check what is possible options for the reason of the acciden**

In [54]:
query = """

SELECT DISTINCT descripcio_causa_mediata
FROM "2022_accidents_vehicles"

"""

result = pd.read_sql_query(query, conn)
result.head()

,descripcio_causa_mediata
0,Manca d'atenció a la conducció
1,Avançament defectuós/improcedent
2,Manca precaució incorporació circulació
3,No determinada
4,No respectar distàncies


**Now I translate to English. I used formula in Excel to write the below repeated lines** 

In [55]:
query = """

 
UPDATE "2022_accidents_vehicles"
SET `descripcio_causa_mediata` = 
    CASE 
        WHEN `descripcio_causa_mediata` = 'Altres' THEN 'others'
        WHEN `descripcio_causa_mediata` = 'No respectar distàncies' THEN 'Not respecting distances'
        WHEN `descripcio_causa_mediata` = "Manca d'atenció a la conducció" THEN 'Lack of attention to driving'
        WHEN `descripcio_causa_mediata` = 'Gir indegut o sense precaució' THEN 'Improper or careless turn'
        WHEN `descripcio_causa_mediata` = 'Manca precaució efectuar marxa enrera' THEN 'Lack of caution in reversing'
        WHEN `descripcio_causa_mediata` = 'No respectat pas de vianants' THEN 'Pedestrian crossing not respected'
        WHEN `descripcio_causa_mediata` = 'Avançament defectuós/improcedent' THEN 'Defective/improper advance'
        WHEN `descripcio_causa_mediata` = 'Desconegut' THEN 'unknown'
        WHEN `descripcio_causa_mediata` = 'Desobeir altres senyals' THEN 'Disobey other signals'
        WHEN `descripcio_causa_mediata` = 'Manca precaució incorporació circulació' THEN 'Lack of caution incorporating circulation'
        WHEN `descripcio_causa_mediata` = 'Canvi de carril sense precaució' THEN 'Lane change without caution'
        WHEN `descripcio_causa_mediata` = 'No determinada' THEN 'not determined'
        WHEN `descripcio_causa_mediata` = 'Desobeir semàfor' THEN 'Disobey traffic lights'
        WHEN `descripcio_causa_mediata` = 'No cedir la dreta' THEN 'Do not yield the right'
        WHEN `descripcio_causa_mediata` = 'Fallada mecànica o avaria' THEN 'Mechanical failure or breakdown'
        WHEN `descripcio_causa_mediata` = 'Envair calçada contrària' THEN 'Invade opposite roadway'
               ELSE `descripcio_causa_mediata`
    END
  WHERE descripcio_causa_mediata IS NOT NULL;
 
 
 
"""

cursor = conn.cursor()
cursor.execute(query)
conn.commit()


**now I validate that description is properly updated**

In [56]:
query = """

SELECT DISTINCT descripcio_causa_mediata
FROM "2022_accidents_vehicles" 
 
"""

result = pd.read_sql_query(query, conn)
result.head()

,descripcio_causa_mediata
0,Lack of attention to driving
1,Defective/improper advance
2,Lack of caution incorporating circulation
3,not determined
4,Not respecting distances


In [58]:
query = """

SELECT *
FROM "2022_accidents_vehicles" 
ORDER BY district_name
"""

result = pd.read_sql_query(query, conn)
result.head()

,case_number,district_code,district_name,neighborhood_code,neighborhood_name,street_code,street_name,postal_code,weekday,year,...,Descripcio_model,Descripcio_marca,Descripcio_color,license_type,license_age,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud,Latitud,descripcio_causa_mediata
0,2022S005521,1,Ciutat Vella,1,el Raval,16408,Àngels ...,0008B0008B,Thursday,2022,...,SES 125,HONDA,Gris,A1,3,430558.97,4581731.11,2.168381,41.382264,others
1,2022S005521,1,Ciutat Vella,1,el Raval,16408,Àngels ...,0008B0008B,Thursday,2022,...,SH 300,HONDA,Blanc,B,Desconegut,430558.97,4581731.11,2.168381,41.382264,others
2,2022S002486,1,Ciutat Vella,1,el Raval,21306,Montserrat ...,0017 0017,Thursday,2022,...,1318,MERCEDES-BENZ,Desconegut,C1,18,431077.50,4581193.61,2.174643,41.377468,unknown
3,2022S004872,1,Ciutat Vella,1,el Raval,21306,Dolors Aleu ...,0041 0041,Thursday,2022,...,TOYOTA PRIUS PLUS,TOYOTA,Altres,B,36,431001.45,4581060.98,2.173749,41.376267,Lack of attention to driving
4,2022S001411,1,Ciutat Vella,1,el Raval,34308,Rambla ...,0015 0015,Tuesday,2022,...,PEOPLE S 125,KYMCO,Gris,B,7,431170.58,4581218.55,2.175753,41.377701,Lack of attention to driving


**Data Clearning, Validation and Manipulation is completed, now I am connecting output file to Tableau for Viz**